# === ETAPA 1: Importações ===

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# === ETAPA 2: Carregamento dos dados ===

In [ ]:
orders = pd.read_csv('olist_orders_dataset.csv', parse_dates=[
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
])
customers = pd.read_csv('olist_customers_dataset.csv')

# Juntando pedidos com localização do cliente
df = pd.merge(orders, customers, on='customer_id', how='left')

# Selecionando colunas de interesse
df = df[[
    'order_purchase_timestamp',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'customer_state'
]].dropna()

# === ETAPA 3: Criação da variável alvo e novas features ===

In [ ]:
df['dias_atraso'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
df['atraso'] = df['dias_atraso'].apply(lambda x: 1 if x > 0 else 0)

df['dias_estimados'] = (df['order_estimated_delivery_date'] - df['order_purchase_timestamp']).dt.days
df['mes_compra'] = df['order_purchase_timestamp'].dt.month


# === ETAPA 4: Preparação dos dados ===

In [ ]:
X = df[['dias_estimados', 'mes_compra', 'customer_state']]
y = df['atraso']

# One-hot encoding para o estado
X = pd.get_dummies(X, columns=['customer_state'], drop_first=True)

# Normalização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


# === ETAPA 5: Treinamento dos modelos ===

In [ ]:
# Regressão Logística
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

# === ETAPA 6: Avaliação dos resultados ===

In [ ]:
print("=== Regressão Logística ===")
print(classification_report(y_test, y_pred_lr))

print("\n=== Random Forest ===")
print(classification_report(y_test, y_pred_rf))

# Matriz de Confusão (Random Forest)
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão - Random Forest')
plt.xlabel('Predição')
plt.ylabel('Valor Real')
plt.show()